In [2]:
import pandas as pd
import psycopg2
import yfinance as yf
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")
load_dotenv()

True

In [3]:
import os
from sqlalchemy import create_engine, text

host = os.environ["SUPABASE_HOST"]
port = os.environ.get("SUPABASE_PORT", 5432)
db   = os.environ["SUPABASE_DB"]
user = os.environ["SUPABASE_USER"]
pw   = os.environ["SUPABASE_PASSWORD"]

url = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{db}?sslmode=require"
engine = create_engine(url, connect_args={"sslmode":"require"}, pool_pre_ping=True)

# 연결 테스트
with engine.begin() as conn:
    print(conn.execute(text("select now()")).scalar())


2025-08-15 13:21:11.365558+00:00


In [4]:
tickers = ["^IXIC", "^GSPC", "^DJI", "^KS11", "KRW=X", "JPYKRW=X", "EURKRW=X", "BTC-USD", "^VIX", "TLT", "^TNX", "^TYX", "GC=F", "SI=F", ]
period = "10y"
interval = "1d"

df = yf.download(tickers, period=period, interval=interval, group_by="ticker", auto_adjust=True)
df = df.stack(level=0)
df = df.reset_index()
df.columns = df.columns.str.lower()

[*********************100%***********************]  14 of 14 completed


In [ ]:
with engine.begin() as conn:
    df.to_sql("stock_data", con=conn, if_exists="replace", index=False)

In [5]:
with engine.begin() as conn:
    df_test = pd.read_sql("SELECT * FROM stock_data", engine)